In [158]:
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd

global spark
spark = (
    pyspark.sql
     .SparkSession
     .builder
     .config("spark.driver.memory", "10g")
     .getOrCreate()
    )
print('Spark version: ', spark.version)

# Reading data:
evidence_parquet = '/Users/dsuveges/project_data/ot/evidence_update/iter13_b/'
df = spark.read.load(evidence_parquet)
df.show()

Spark version:  3.0.0
+---------------------------------+---------+-------------------+-------------------------------------+----------------------------+--------------+--------------+----------------+------+------------------------------+---------------+----------------+-------------+---------------+---------+----------+---------+-------------------+----------+-------------------------------------+----------------------+--------------+--------------+---------------+----------------------------+-------------+--------------------+--------+------------------+------------+------------------------+----------+---------------------+----------+--------------------------------+--------------------+----------------+-------------------+-----------------------+--------+-------------+-----------------------+-------------+-----------+-----------------+-----------+----------+------------+---------------+-------+---------------+--------------------+--------------------+---------+
|biologicalModelAlle

In [170]:
notnull = []

for column in df.columns:
    sources = not_null = (
        df
        .filter(f'{column} is not null')
        .select('sourceId')
        .distinct()
        .toPandas()
    )
    
    notnull.append({
        'column': column,
        'sources': ','.join(sources.sourceId.tolist())
    })
    
sources_df = pd.DataFrame(notnull)
sources_df

,column,sources
0,biologicalModelAllelicComposition,phenodigm
1,variantId,"phewas_catalog,ot_genetics_portal"
2,diseaseFromSourceId,"europepmc,crispr,genomics_england,chembl,slape..."
3,diseaseModelAssociatedModelPhenotypes,phenodigm
4,variantAminoacidDescriptions,reactome
5,clinicalStatus,chembl
6,mutatedSamples,"intogen,cancer_gene_census"
7,cohortPhenotypes,genomics_england
8,drugId,chembl
9,variantFunctionalConsequenceId,"phewas_catalog,eva,ot_genetics_portal"


In [190]:
sources_df.sort_values('column', inplace=True)
sources_df.to_csv('/Users/dsuveges/project/random_notebooks/issue-1249_prototyping_new_json_schema/fields_sources.tsv', index=False, sep='\t')


In [179]:
mutated_samples = (
    df
    .filter('mutatedSamples is not null')
    .select(explode("mutatedSamples").alias("exploded"))
    .select("exploded.*")
    .toPandas()
)
mutated_samples.head()


,functionalConsequenceId,numberMutatedSamples,numberSamplesTested,numberSamplesWithMutationType
0,SO_0001605,1240,9180,1.0
1,SO_0001059,1240,9180,44.0
2,SO_0001589,1240,9180,1.0
3,SO_0001583,1240,9180,1194.0
4,SO_0001059,6,626,6.0


In [177]:
mutated_samples

,mutatedSamples
0,"[(SO_0001605, 1240, 9180, 1), (SO_0001059, 124..."
1,"[(SO_0001059, 6, 626, 6), (SO_0001583, 6, 626,..."
2,"[(SO_0001587, 22, 802, 1), (SO_0001059, 22, 80..."
3,"[(SO_0001589, 3, 455, 1), (SO_0001583, 3, 455,..."
4,"[(SO_0001059, 14, 370, 1), (SO_0001583, 14, 37..."
...,...
65003,"[(None, 3, 121, None)]"
65004,"[(None, 13, 58, None)]"
65005,"[(None, 2, 148, None)]"
65006,"[(None, 3, 15, None)]"


In [188]:
df.select(sorted(df.columns)).printSchema()

root
 |-- allelicRequirements: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- biologicalModelAllelicComposition: string (nullable = true)
 |-- biologicalModelGeneticBackground: string (nullable = true)
 |-- clinicalPhase: long (nullable = true)
 |-- clinicalSignificances: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- clinicalStatus: string (nullable = true)
 |-- clinicalUrls: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- niceName: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- cohortDescription: string (nullable = true)
 |-- cohortId: string (nullable = true)
 |-- cohortPhenotypes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- cohortShortName: string (nullable = true)
 |-- confidence: string (nullable = true)
 |-- confidenceIntervalLower: double (nullable = true)
 |-- confidenceIntervalUpper: double (nullable = true)
 |-- contr

In [196]:
(
    df
    .filter('variantAminoacidDescriptions is not null')
    .select('variantAminoacidDescriptions','sourceId')
    .show()
)

+----------------------------+--------+
|variantAminoacidDescriptions|sourceId|
+----------------------------+--------+
|        [L-leucine 389 re...|reactome|
|        [L-glutamic acid ...|reactome|
|                          []|reactome|
|        [L-isoleucine 161...|reactome|
|                          []|reactome|
|        [L-threonine 680 ...|reactome|
|        [L-tyrosine 207 r...|reactome|
|        [Insertion of res...|reactome|
|                          []|reactome|
|                          []|reactome|
|                          []|reactome|
|                          []|reactome|
|        [glycine 12 repla...|reactome|
|                          []|reactome|
|                          []|reactome|
|        [L-valine 1576 re...|reactome|
|                          []|reactome|
|        [L-asparagine 822...|reactome|
|        [L-proline 252 re...|reactome|
|        [L-glutamine 61 r...|reactome|
+----------------------------+--------+
only showing top 20 rows



In [202]:
sources_df.loc[sources_df.column == 'dataTypeId']

,column,sources


In [ ]:
sources_df